In [0]:
from pyspark.sql import functions as fn
from pyspark.sql.functions import col, lit, isnan, when, count
from pyspark.sql.types import *
from pyspark.sql import Window

In [0]:
df = spark.read.csv("s3://geekbrains-spark/sf-fire-calls.csv", header=True)

In [0]:
display(df.limit(5))

Call Number,Unit ID,Incident Number,CallType,Call Date,Watch Date,Call Final Disposition,Available DtTm,Address,City,Zipcode of Incident,Battalion,Station Area,Box,OrigPriority,Priority,Final Priority,ALS Unit,Call Type Group,NumAlarms,UnitType,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhood,Location,RowID,Delay
20110014,M29,2003234,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 01:58:43 AM,10TH ST/MARKET ST,SF,94103,B02,36,2338,1,1,2,true,null,1,MEDIC,1,2,6,Tenderloin,"(37.7765408927183, -122.417501464907)",020110014-M29,5.233333333333333
20110015,M08,2003233,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:10:17 AM,300 Block of 5TH ST,SF,94107,B03,08,2243,1,1,2,true,null,1,MEDIC,1,3,6,South of Market,"(37.7792841462441, -122.402061300134)",020110015-M08,3.0833333333333335
20110016,B02,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,6,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B02,3.05
20110016,B04,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:54 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,3,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B04,2.316666666666667
20110016,D2,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,4,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-D2,3.0166666666666666


Переименуем колонку с датой вызова и из нее сделаем колонки с датой в формате datetime  и еще отдельно год, месяц, день

In [0]:
df = df.withColumnRenamed("Call Date", "Call_Date_str")

In [0]:
df = df.withColumn("call_date", fn.to_date("Call_Date_str", "dd/MM/yyyy"))

In [0]:
df = df.withColumn("call_date_year", fn.year(df.call_date)) \
       .withColumn("call_date_month", fn.month(df.call_date)) \
       .withColumn("call_date_day", fn.dayofmonth(df.call_date))

In [0]:
display(df.limit(5))

Call Number,Unit ID,Incident Number,CallType,Call_Date_str,Watch Date,Call Final Disposition,Available DtTm,Address,City,Zipcode of Incident,Battalion,Station Area,Box,OrigPriority,Priority,Final Priority,ALS Unit,Call Type Group,NumAlarms,UnitType,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhood,Location,RowID,Delay,call_date,call_date_year,call_date_month,call_date_day
20110014,M29,2003234,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 01:58:43 AM,10TH ST/MARKET ST,SF,94103,B02,36,2338,1,1,2,true,null,1,MEDIC,1,2,6,Tenderloin,"(37.7765408927183, -122.417501464907)",020110014-M29,5.233333333333333,2002-11-01,2002,11,1
20110015,M08,2003233,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:10:17 AM,300 Block of 5TH ST,SF,94107,B03,08,2243,1,1,2,true,null,1,MEDIC,1,3,6,South of Market,"(37.7792841462441, -122.402061300134)",020110015-M08,3.0833333333333335,2002-11-01,2002,11,1
20110016,B02,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,6,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B02,3.05,2002-11-01,2002,11,1
20110016,B04,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:54 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,3,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B04,2.316666666666667,2002-11-01,2002,11,1
20110016,D2,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,4,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-D2,3.0166666666666666,2002-11-01,2002,11,1


Убираем из имен колонок пробелы

In [0]:
df = df.select([fn.col(col_name).alias(col_name.replace(' ', '_')) for col_name in df.columns])

In [0]:
display(df.limit(5))

Call_Number,Unit_ID,Incident_Number,CallType,Call_Date_str,Watch_Date,Call_Final_Disposition,Available_DtTm,Address,City,Zipcode_of_Incident,Battalion,Station_Area,Box,OrigPriority,Priority,Final_Priority,ALS_Unit,Call_Type_Group,NumAlarms,UnitType,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhood,Location,RowID,Delay,call_date,call_date_year,call_date_month,call_date_day
20110014,M29,2003234,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 01:58:43 AM,10TH ST/MARKET ST,SF,94103,B02,36,2338,1,1,2,true,null,1,MEDIC,1,2,6,Tenderloin,"(37.7765408927183, -122.417501464907)",020110014-M29,5.233333333333333,2002-11-01,2002,11,1
20110015,M08,2003233,Medical Incident,01/11/2002,01/10/2002,Other,01/11/2002 02:10:17 AM,300 Block of 5TH ST,SF,94107,B03,08,2243,1,1,2,true,null,1,MEDIC,1,3,6,South of Market,"(37.7792841462441, -122.402061300134)",020110015-M08,3.0833333333333335,2002-11-01,2002,11,1
20110016,B02,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,6,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B02,3.05,2002-11-01,2002,11,1
20110016,B04,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:51:54 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,3,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-B04,2.316666666666667,2002-11-01,2002,11,1
20110016,D2,2003235,Structure Fire,01/11/2002,01/10/2002,Other,01/11/2002 01:47:00 AM,2000 Block of CALIFORNIA ST,SF,94109,B04,38,3362,3,3,3,false,null,1,CHIEF,4,4,5,Pacific Heights,"(37.7895840679362, -122.428071912459)",020110016-D2,3.0166666666666666,2002-11-01,2002,11,1


Сделаем запрос, чтобы ответить 

Какие типы вызовов случались в 2018? Перечислите все.

In [0]:
display(
  df
  .filter(col("call_date_year") == 2018)
  .select(col("CallType"))
  .distinct()
)

CallType
Elevator / Escalator Rescue
Confined Space / Structure Collapse
Administrative
Alarms
Odor (Strange / Unknown)
Citizen Assist / Service Call
HazMat
Explosion
Vehicle Fire
Suspicious Package


Видим, что запрос. выполняется очень долго.

Попробуем пересохранить датафрейм в parquet формат с партиционированием по году

и затем снова выполним запрос

сохраняем в паркет

In [0]:
df.write.partitionBy("call_date_year").parquet("./firecalls.parquet")

In [0]:
%fs ls /

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/firecalls.parquet/,firecalls.parquet/,0


In [0]:
%fs ls /firecalls.parquet

path,name,size
dbfs:/firecalls.parquet/_SUCCESS,_SUCCESS,0
dbfs:/firecalls.parquet/call_date_year=2000/,call_date_year=2000/,0
dbfs:/firecalls.parquet/call_date_year=2001/,call_date_year=2001/,0
dbfs:/firecalls.parquet/call_date_year=2002/,call_date_year=2002/,0
dbfs:/firecalls.parquet/call_date_year=2003/,call_date_year=2003/,0
dbfs:/firecalls.parquet/call_date_year=2004/,call_date_year=2004/,0
dbfs:/firecalls.parquet/call_date_year=2005/,call_date_year=2005/,0
dbfs:/firecalls.parquet/call_date_year=2006/,call_date_year=2006/,0
dbfs:/firecalls.parquet/call_date_year=2007/,call_date_year=2007/,0
dbfs:/firecalls.parquet/call_date_year=2008/,call_date_year=2008/,0


Видим что создался паркет с партициями по годам

Создаем датафрейм из паркет-файла

In [0]:
parq_df = spark.read.parquet("dbfs:/firecalls.parquet/")

Снова выполним тот же самый запрос:

Какие типы вызовов случались в 2018? Перечислите все.

In [0]:
display(
  parq_df
  .filter(col("call_date_year") == 2018)
  .select(col("CallType"))
  .distinct()
)

CallType
Elevator / Escalator Rescue
Confined Space / Structure Collapse
Alarms
Odor (Strange / Unknown)
Citizen Assist / Service Call
HazMat
Explosion
Vehicle Fire
Suspicious Package
"Extrication / Entrapped (Machinery, Vehicle)"


Видим что время запроса уменьшилось с 2х минут до почти 2х секунд.

Дальше запросы будем делать уже к этому датафрейму.

В каком месяце 2018 года было больше всего вызовов?

In [0]:
display(
  parq_df
  .filter(col("call_date_year") == 2018)
  .groupby("call_date_month")
  .count()
  .sort(col("count").desc())
  .limit(1) 
)

call_date_month,count
1,9591


Из какого района (neighborhood) было больше всего вызовов в 2018?

In [0]:
display(
  parq_df
  .filter(col("call_date_year") == 2018)
  .groupby("Neighborhood")
  .count()
  .sort(col("count").desc())
  .limit(1) 
)

Neighborhood,count
Tenderloin,14834


Какие районы (neighborhoods) дольше всего ждали (delay) пожарных в 2018?

In [0]:
display(
  parq_df
  .filter(col("call_date_year") == 2018)
  .groupby("Neighborhood")
  .max("Delay")
)

---------------------------------------------------------------------------
AnalysisException Traceback (most recent call last)
 in 
 1 display(
----> 2 parq_df
 3 .filter(col("call_date_year") == 2018)
 4 .groupby("Neighborhood")
 5 .max("Delay")

/databricks/spark/python/pyspark/sql/group.py in _api(self, *cols)
 39 def _api(self, *cols):
 40 name = f.__name__
---> 41 jdf = getattr(self._jgd, name)(_to_seq(self.sql_ctx._sc, cols))
 42 return DataFrame(jdf, self.sql_ctx)
 43 _api.__name__ = f.__name__

/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__(self, *args)
 1302 
 1303 answer = self.gateway_client.send_command(command)
-> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
 121 # Hide where the exception came from that shows a non-Pythonic
 122 # JVM exception message.
--> 123 raise converted from None
 124 else:
 125 raise

AnalysisException: "Delay" is not a numeric column. Aggregation function can only be applied on a numeric column.

Нужно изменить тип колонки Delay

In [0]:
from pyspark.sql.types import DoubleType
parq_df = parq_df.withColumn("Delay",parq_df.Delay.cast(DoubleType()))

In [0]:
parq_df.printSchema()

root
-- Call_Number: string (nullable = true)
-- Unit_ID: string (nullable = true)
-- Incident_Number: string (nullable = true)
-- CallType: string (nullable = true)
-- Call_Date_str: string (nullable = true)
-- Watch_Date: string (nullable = true)
-- Call_Final_Disposition: string (nullable = true)
-- Available_DtTm: string (nullable = true)
-- Address: string (nullable = true)
-- City: string (nullable = true)
-- Zipcode_of_Incident: string (nullable = true)
-- Battalion: string (nullable = true)
-- Station_Area: string (nullable = true)
-- Box: string (nullable = true)
-- OrigPriority: string (nullable = true)
-- Priority: string (nullable = true)
-- Final_Priority: string (nullable = true)
-- ALS_Unit: string (nullable = true)
-- Call_Type_Group: string (nullable = true)
-- NumAlarms: string (nullable = true)
-- UnitType: string (nullable = true)
-- Unit_sequence_in_call_dispatch: string (nullable = true)
-- Fire_Prevention_District: string (nullable = true)
-- Supervisor_District: string (nullable = true)
-- Neighborhood: string (nullable = true)
-- Location: string (nullable = true)
-- RowID: string (nullable = true)
-- Delay: double (nullable = true)
-- call_date: date (nullable = true)
-- call_date_month: integer (nullable = true)
-- call_date_day: integer (nullable = true)
-- call_date_year: integer (nullable = true)

Выполним запрос еще раз

In [0]:
display(
  parq_df
  .filter(col("call_date_year") == 2018)
  .groupby("Neighborhood")
  .agg(fn.max("Delay").alias("max_delay"))
  .sort(col("max_delay").desc())
)

Neighborhood,max_delay
Bayview Hunters Point,715.7666666666667
Sunset/Parkside,457.4
Potrero Hill,336.65
Outer Mission,290.3833333333333
Treasure Island,266.46666666666664
Pacific Heights,196.1
Russian Hill,192.95
Mission,169.25
Financial District/South Beach,135.51666666666668
South of Market,129.31666666666666


Какая неделя в 2018 году была самая активная по количеству вызовов?

In [0]:
from pyspark.sql.functions import weekofyear
display(
  parq_df
  .withColumn("week_num", weekofyear(parq_df.call_date))
  .filter(col("call_date_year") == 2018)
  .groupby("week_num")
  .count()
  .sort(col("count").desc())
  .limit(1)
)

week_num,count
1,6276
